In [ ]:
!pip install bing_image_downloader

In [ ]:
!rm -rf image

Taking input


In [ ]:
from bing_image_downloader import downloader
downloader.download("motorcycle" , output_dir = "images" , limit = 30)
downloader.download("car" , output_dir = "images" , limit = 30)

[%] Downloading Images to /content/images/motorcycle


[!!]Indexing page: 1

[%] Indexed 30 Images on Page 1.


[%] Downloading Image #1 from https://i.pinimg.com/originals/00/6b/8c/006b8c48b868d930eedf3cb04f2fed59.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://i.pinimg.com/originals/d2/f3/88/d2f388798c7d18f3a9d99bdded416778.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://uploads.tapatalk-cdn.com/20201031/7978c3d16127bebd58d7ff5bf808012a.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://i.pinimg.com/originals/6c/19/59/6c1959695987a21666a31184249c6616.png
[%] File Downloaded !

[%] Downloading Image #5 from https://i.pinimg.com/originals/3c/7c/37/3c7c3789a297df5c18b63b3293e05614.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://blog.motorcycle.com/wp-content/uploads/2018/03/032318-True-American-Heroes-Weekend-XSR700.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://i.pinimg.com/originals/35/f0/06/35f00659537

Importing libraries

In [ ]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt

Preprocessing

In [ ]:
Categories=['motorcycle','car']
flat_data_arr=[] #input array
target_arr=[] #output array
DATADIR =  "/content/images"
for i in Categories:
    path=os.path.join(DATADIR,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(150,150,3))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) 
df['Target']=target
X=df.iloc[:,:-1] 
y=df.iloc[:,-1] 

Model construction

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,stratify=y)

leaf_size = [i for i in range(1,20)]
n_neighbors = [i for i in range(1,10)]
p=[1,2]
#Convert to dictionary
hyperparameters = {'leaf_size' : leaf_size, 'n_neighbors': n_neighbors, 'p' : p}
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
#Fit the model
best_model = clf.fit(X_train,y_train)

Score:


In [ ]:
y_pred=best_model.predict(X_test)
best_model.score(X_test , y_test)

PREDICT:

In [ ]:
url=input('Enter URL of Image :')
img=imread(url)
plt.imshow(img)
plt.show()
img_resize=resize(img,(150,150,3))
img_=[img_resize.flatten()]
probability=best_model.predict_proba(img_)
for ind,val in enumerate(Categories):
    print(f'{val} = {probability[0][ind]*100}%')
print("The predicted image is : "+Categories[best_model.predict(img_)[0]])


Save model

In [ ]:
import pickle
pickle.dump(best_model , open("model.pkl","wb"))

